# Whence cometh the L in LU?

Last time, we constructed the LU factorization by what may have seemed like a laborious procedure.  Getting $U$ was "easy", it was just Gaussian elimination.  But to get $L$, we first wrote out the individual elimination steps as matrices, then inverted them to move them to the other side, then multiplied them together to get $L$.

However, it turns out that we can just "read off" $L$ much more simply directly from the pivot-row "multipliers" that we use during elimination steps.  To see this, let's first write a Julia function to perform Gaussian elimination (without row swaps!) and print out all of the steps, adapting our function from pset 1:

In [1]:
# perform Gaussian elimination of A without row swaps, returning U,
# while printing a message for each elimination step.
function print_gauss(A)
    m = size(A,1) # number of rows
    U = copy!(similar(A, typeof(inv(A[1,1]))), A)
    for j = 1:m   # loop over m columns
        for i = j+1:m   # loop over rows below the pivot row j
            # subtract a multiple of the pivot row (j)
            # from the current row (i) to cancel U[i,j] = Uᵢⱼ:
            ℓᵢⱼ = U[i,j]/U[j,j]
            println("subtracting $ℓᵢⱼ × (row $j) from (row $i)")
            U[i,:] = U[i,:] - U[j,:] * ℓᵢⱼ
            U[i,j] = 0 # store exact zero to compensate for roundoff errors
        end
    end
    return U
end

print_gauss (generic function with 1 method)

Now, let's try it on a randomly chosen $5 \times 5$ matrix $A$:

In [2]:
A = [4  -2  -7  -4  -8
     9  -6  -6  -1  -5
    -2  -9   3  -5   2
     9   7  -9   5  -8
    -1   6  -3   9   6]

5×5 Array{Int64,2}:
  4  -2  -7  -4  -8
  9  -6  -6  -1  -5
 -2  -9   3  -5   2
  9   7  -9   5  -8
 -1   6  -3   9   6

In [3]:
print_gauss(A)

subtracting 2.25 × (row 1) from (row 2)
subtracting -0.5 × (row 1) from (row 3)
subtracting 2.25 × (row 1) from (row 4)
subtracting -0.25 × (row 1) from (row 5)
subtracting 6.666666666666667 × (row 2) from (row 3)
subtracting -7.666666666666667 × (row 2) from (row 4)
subtracting -3.6666666666666665 × (row 2) from (row 5)
subtracting -1.2442748091603053 × (row 3) from (row 4)
subtracting -0.4732824427480916 × (row 3) from (row 5)
subtracting 33.495145631067295 × (row 4) from (row 5)


5×5 Array{Float64,2}:
 4.0  -2.0   -7.0    -4.0        -8.0     
 0.0  -1.5    9.75    8.0        13.0     
 0.0   0.0  -65.5   -60.3333    -88.6667  
 0.0   0.0    0.0     0.262087   -0.659033
 0.0   0.0    0.0     0.0        31.7767  

In comparison, here is the LU factorization of $A$ from the built-in `lu` function, with row-swaps disabled:

In [4]:
L, U = lu(A, Val{false})
U

5×5 Array{Float64,2}:
 4.0  -2.0   -7.0    -4.0        -8.0     
 0.0  -1.5    9.75    8.0        13.0     
 0.0   0.0  -65.5   -60.3333    -88.6667  
 0.0   0.0    0.0     0.262087   -0.659033
 0.0   0.0    0.0     0.0        31.7767  

Same $U$ matrix!  Now let's look at $L$:

In [5]:
L

5×5 Array{Float64,2}:
  1.0    0.0       0.0        0.0     0.0
  2.25   1.0       0.0        0.0     0.0
 -0.5    6.66667   1.0        0.0     0.0
  2.25  -7.66667  -1.24427    1.0     0.0
 -0.25  -3.66667  -0.473282  33.4951  1.0

Notice that the entries of $L$ below the diagonal are *exactly* the multipliers that were printed out during Gaussian elimination (the factors by which the pivot row is multiplied before it is *subtracted* from a row below it).

One way to see this is to consider the matrix product $LU$, which should give $A$.  Consider, for example, the third row of this: the third row of $L$ tells us what linear combinations of the rows of $U$ gives the third row of $A$.  It says:

* third row of `A` = `[-2,-9,3,-5,2]` = `-0.5` × (row 1 of U) + 6.66... × (row 2 of U) + (row 3 of U)

In [6]:
L[3,1] * U[1,:] + L[3,2] * U[2,:] + U[3,:]

5-element Array{Float64,1}:
 -2.0
 -9.0
  3.0
 -5.0
  2.0

But this is exactly the *reverse of the elimination steps*, so of course it works.  **Putting the multipliers in $L$ is the right thing!**

See section 2.6 of the textbook for more info.

Still, computing the $L$ in the LU factorization requires care to put all of the multipliers in the right place with the right sign.  It is a pain for human beings, which is why we typically don't do it when performing Gaussian elimination by hand.  However, computers are great at this kind of tedious bookkeeping, and since keeping track of $L$ requires almost *no extra work*, computers essentially *always* figure out *both* $L$ and $U$ when doing Gaussian elimination.

# Using LU factorizations

Lots of things that you might want to do with a matrix $A$ become easier once you have the $A=LU$ factorization.  Most importantly, it becomes much easier to solve systems of equations.

(Exactly *how much* easier is something we'll quantify later.  Short answer: for an $n \times n$ matrix $A$, it takes around $n^3$ operations to perform Gaussian elimination to get $U$ and $L$, but subsequently solving for $x$ by takes only around $n^2$ operations.)

## Solving Ax=b

When we do Gaussian elimination by hand, we convert $Ax = b$ to $Ux = c$ by performing the same elimination steps on $b$ to get $c$ as we performed on $A$ to get $U$.  Often, we do this by "augmenting" the matrix $A$ with the right-hand side $b$.  This makes it easier (*for hand calculation*) to keep track of what operations to do on $b$. For example:

In [7]:
b = rand(-9:9, 5)

5-element Array{Int64,1}:
 -9
 -2
  3
  5
  6

In [8]:
_, U_and_c = lu([A b], Val{false}) # eliminate augmented matrix (without row swaps)
U = UpperTriangular(U_and_c[:, 1:end-1]) # all but last column is U
c = U_and_c[:, end] # last column is c

5-element Array{Float64,1}:
   -9.0   
   18.25  
 -123.167 
   11.9135
 -386.67  

Then we can solve $Ux = c$ by backsubstitution (`U \ c`), and it should give the same answer (up to roundoff error) as `A \ b`:

In [9]:
[U\c  A\b] # print them side by side

5×2 Array{Float64,2}:
  -7.58753   -7.58753
  -8.05041   -8.05041
   4.66636    4.66636
  14.8582    14.8582 
 -12.1683   -12.1683 

However, the computer doesn't do this: on a computer, you **almost never augment the matrix with the right-hand-side**.  Instead, you:

1. Factor $A = LU$ by Gaussian elimination (not including row swaps, discussed below!), giving $Ax = b \implies LUx = L(Ux) = b$
2. Let $c = Ux$.  Solve $Lc = b$ for $c$ by forward-substitution.
3. Solve $Ux = c$ for $x$ by backsubstitution.

The key point to realize is that solving $Lc = b$ for $c$ involves *exactly the same elimination steps* as if you had augmented the matrix with $b$ during Gaussian elimination.   The bookkeeping is more tedious for a human, but computers are good at bookkeeping, and there turn out to be several practical advantages for computer software to separate solving for $LU$ and solving for $c$.

In [10]:
L, U = lu(A, Val{false}) # Gaussian elimination without row swaps
c = L \ b # solve Lc = b for c

5-element Array{Float64,1}:
   -9.0   
   18.25  
 -123.167 
   11.9135
 -386.67  

Same $c$ as before!

Let's write a little program to write out the steps of forward-substitution so that we can see that they are indeed the elimination steps from before:

In [11]:
c = similar(b, Float64)
for i = 1:length(b)
    print("c[$i] = b[$i]")
    c[i] = b[i]
    for j = 1:i-1
        print("- $(L[i,j]) * c[$j]")
        c[i] = c[i] - L[i,j] * c[j]
    end
    println(" = ", c[i])
end
c

c[1] = b[1] = -9.0
c[2] = b[2]- 2.25 * c[1] = 18.25
c[3] = b[3]- -0.5 * c[1]- 6.666666666666666 * c[2] = -123.16666666666666
c[4] = b[4]- 2.25 * c[1]- -7.666666666666666 * c[2]- -1.2442748091603053 * c[3] = 11.913486005089055
c[5] = b[5]- -0.25 * c[1]- -3.6666666666666665 * c[2]- -0.47328244274809156 * c[3]- 33.495145631067324 * c[4] = -386.6699029126136


5-element Array{Float64,1}:
   -9.0   
   18.25  
 -123.167 
   11.9135
 -386.67  

In Julia, `A \ b` does this whole process for you implicitly.

## Multiple right-hand sides and AX = B

Suppose that we need to solve $Ax=b$ for **multiple right-hand sides** $b_1$, $b_2$, and so on.   Once we have computed $A=LU$ by Gaussian elimination, we can *re-use* $L$ and $U$ to solve each new right-hand side:

1. Find $A = LU$ by Gaussian elimination
2. Solve $Ax_1 = b_1$ by `x₁ = U \ (L \ b₁)`
3. Solve $Ax_1 = b_2$ by `x₂ = U \ (L \ b₂)`
4. etcetera

Since solving triangular systems of equations ($L$ or $U$) is easy, this way we only do the hard/expensive part (Gaussian elimination once).

Julia provides a shorthand for this process, so you don't have to worry about $L$ and $U$ and explicit forward/backsubstitution.  Instead, you compute `LU = lufact(A)`, which creates an "LU factorization object" `LU` that internally stores $L$ and $U$ in a compressed format (along with any permutations/row swaps as discussed below), and then you can do `LU \ b` for each new right-hand side and it will do the (fast) triangular solves:

In [12]:
LU = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}}([4.0 -2.0 … -4.0 -8.0; 2.25 -1.5 … 8.0 13.0; … ; 2.25 -7.66667 … 0.262087 -0.659033; -0.25 -3.66667 … 33.4951 31.7767],[1,2,3,4,5],0)

In [13]:
[LU\b  A\b] # print them side by side

5×2 Array{Float64,2}:
  -7.58753   -7.58753
  -8.05041   -8.05041
   4.66636    4.66636
  14.8582    14.8582 
 -12.1683   -12.1683 

# Row swaps and PA = LU

Up to now, we have mostly ignored the possibility of row swaps.  Row swaps may be *required* if you encounter a zero pivot (assuming there is a nonzero value below it in the same column), but this is often unlikely to occur in practice (especially for random matrices!).

However, even as in the example above where no row swaps were *required*, a computer will often do them *anyway*, in order to minimize roundoff errors.  As you saw on pset 1, roundoff errors (the computer only keeps about 15–16 significant digits) can be disastrous if the pivot is merely very small.  So, the computer swaps rows to **make the pivot as big as possible**, as strategy called *partial pivoting*.  As a result, the `lu` function in Julia returns *three* things: $L$, $U$, and the permutation $p$ giving the **re-ordering of the rows** of $A$ that is needed.  For example:

In [14]:
L, U, p = lu(A)
L

5×5 Array{Float64,2}:
  1.0        0.0        0.0       0.0        0.0
  1.0        1.0        0.0       0.0        0.0
  0.444444   0.0512821  1.0       0.0        0.0
 -0.111111   0.410256   0.582822  1.0        0.0
 -0.222222  -0.794872   0.171779  0.0242696  1.0

In [15]:
U

5×5 Array{Float64,2}:
 9.0  -6.0  -6.0      -1.0      -5.0     
 0.0  13.0  -3.0       6.0      -3.0     
 0.0   0.0  -4.17949  -3.86325  -5.62393 
 0.0   0.0   0.0       8.67894   9.95297 
 0.0   0.0   0.0       0.0      -0.771206

In [16]:
p

5-element Array{Int64,1}:
 2
 4
 1
 5
 3

`p` says tells you in what order we should put the rows of $A$ to match the product $LU$: we should re-order `A` to put row 2 first, then row 4, then row 1, then row 5, then row 3.  We can do this in Julia easily by:

In [17]:
A[p,:] # A with the rows in order p

5×5 Array{Int64,2}:
  9  -6  -6  -1  -5
  9   7  -9   5  -8
  4  -2  -7  -4  -8
 -1   6  -3   9   6
 -2  -9   3  -5   2

This should match $LU$:

In [18]:
L*U

5×5 Array{Float64,2}:
  9.0  -6.0  -6.0  -1.0  -5.0
  9.0   7.0  -9.0   5.0  -8.0
  4.0  -2.0  -7.0  -4.0  -8.0
 -1.0   6.0  -3.0   9.0   6.0
 -2.0  -9.0   3.0  -5.0   2.0

The computer only stores a list of numbers for `p` because that is the most efficient way to store and work with the permutation.  However, for algebraic manipulations it is often convenient to think of this as a **permutation matrix** $P$ multiplying $A$.  Since $P$ re-orders the *rows* of $A$, it must multiply $A$ on the *left*.  Constructing $P$ is easy: it just has a single 1 in each row indicating what row of $A$ should go there.

In [19]:
# construct a permutation matrix P from the permutation vector p
function permutation_matrix(p)
    P = zeros(Int, length(p),length(p))
    for i = 1:length(p)
        P[i,p[i]] = 1
    end
    return P
end

permutation_matrix (generic function with 1 method)

In [20]:
P = permutation_matrix(p)

5×5 Array{Int64,2}:
 0  1  0  0  0
 0  0  0  1  0
 1  0  0  0  0
 0  0  0  0  1
 0  0  1  0  0

In [21]:
P * A == A[p, :]

true

Thus, LU factorization with row swaps corresponds to a factorization

$$
PA = LU
$$

Now, to solve $Ax = b$, a more complete process is:

1. Factor $PA = LU$
2. Multiply $P$ by both sides to give $PAx = LUx = Pb$
3. Let $c=Ux$ and solve $Lc = Pb$ for $c$ by forward-substitution
3. Solve $Ux = c$ for $x$ by backsubstitution.

Of course, Julia does all of this for you automatically with `A \ b` or `lufact(A) \ b`, but we can do it manually:

In [24]:
c = L \ b[p] # solve Lc = Pb = b[p]
x = U \ c # solve Ux = c

5-element Array{Float64,1}:
  -7.58753
  -8.05041
   4.66636
  14.8582 
 -12.1683 

Hooray, this is the same answer as above!

One final point often confuses people here, if you think carefully about the above process.   By writing $PA = LU$, it seems like you *first* decide on the row-reordering of $A$, and *then* compute the LU factorization of $PA$.  But how do you know the proper row-reordering *before* you do elimination?  In fact, this is an illusion: the computer figures out the row-reordering as it goes along (partial pivoting as described above), but it then cleverly works backwards to figure out what reordering it *should* have done in the beginning!

# Singular matrices

# Matrix inverses